<a href="https://colab.research.google.com/github/Abhishekkohli/Abhishek27/blob/master/Copy_of_internshala_question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from time import ctime
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from tqdm import tqdm

In [0]:
pip install stockstats

In [0]:
pip install yfinance

In [0]:
from stockstats import StockDataFrame

In [0]:
#finding today's time and day
s=ctime()

In [0]:
day=s.split(' ',1)[0]
i=s.find(':')
#since time is computed for GMT zone, so converting it in our IST zone
hour=(int(s[i-2:i]) + 6)%24
minute=int(s[i+1:i+3]) - 30
if minute < 0 :
  minute+=60
  hour-=1
total_time_minutes=hour*60 + minute

In [0]:

url = "https://www.moneycontrol.com/stocks/marketstats/indexcomp.php?optex=NSE&opttopic=indexcomp&index=9"

reponse = get(url)
html_soup = bs(reponse.text,'html.parser')
table_element = html_soup.find('table',attrs={'class':'tbldata14 bdrtpg'})
table_rows = table_element.find_all('tr')
companies = table_rows[1:]
#to store previous %chg values for companies in list
previous_change = []
#to keep track of time
program_time = 0

final_array = []
#to calculate modulus
def mod(a):
    if(a < 0):
        return -a
    return a

#to alert when %chg crosses 2
def check_alert(previous_data,new_data,stock_name):
    if(mod(new_data-previous_data)>=2):
        print("ALERT!!  "+ stock_name +" has more than 2 percent change")

#storing table in dataframe
for i in tqdm(range(len(companies))):
    local_array = []
    company = companies[i]
    tds = company.find_all('td')
    name = tds[0].find('a',attrs={'class':'bl_12'}).text
    industry = tds[1].find('a',attrs={'class':'bl_12'}).text
    last_price = tds[2].text
    change = tds[3].text
    change_percent = tds[4].text
    market_cap = tds[5].text
    previous_change.append(change_percent)
    local_array.append(name)
    local_array.append(industry)
    local_array.append(last_price)
    local_array.append(change)
    local_array.append(change_percent)
    local_array.append(market_cap)
    final_array.append(local_array)

df=pd.DataFrame(final_array,columns=["Company_Name","Industry","Last_Price","Change","%chg","Market_cap(in crs.)"])
print()


100%|██████████| 50/50 [00:00<00:00, 8757.11it/s]

In [0]:
def update_array():
    global df
    for i in tqdm(range(len(companies))):
        company = companies[i]
        tds = company.find_all('td')
        name = tds[0].find('a',attrs={'class':'bl_12'}).text
        industry = tds[1].find('a',attrs={'class':'bl_12'}).text
        last_price = tds[2].text
        change = tds[3].text
        change_percent = tds[4].text
        #checking alert at every 2 minutes  
        if(program_time%120 == 0):
            check_alert(float(previous_change[i]),float(change_percent),name)
        previous_change[i]=change_percent
        market_cap = tds[5].text
        df["Company_Name"][i]=name
        df["Industry"][i]=industry
        df["Last_Price"][i]=last_price
        df["Change"][i]=change
        df["%chg"][i]=change_percent
        df["Market_cap(in crs.)"][i]=market_cap

t1=9*60 + 15
t2=15*60 + 30
while True:
    time.sleep(30)
    program_time+=30
    # since real table is updated only during Mon-Fri and between 9:15 and 15:30, so we are updating our table in the given time period
    if (day!="Sat" and day!="Sun") and (t1 <=total_time_minutes<=t2):
        update_array()
    print(df.head())
    #program running for 20 minutes
    if(program_time == 1200):
        break



In [0]:
import yfinance as yf

#define the ticker symbol for SBI, which is there in the table
tickerSymbol = 'SBIN.NS'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
df_stock = tickerData.history(period='1d', start='2020-3-1', end='2020-3-16')
df_stock.head()

In [0]:
#implementing Relative Strength Index (RSI) as an indicator for 14 days
stock=StockDataFrame.retype(df_stock)
df_stock['rsi']=stock['rsi_14']
#removing extra columns
del df_stock['closenm_14_smma']
del df_stock['closepm_14_smma']
del df_stock['closenm']
del df_stock['closepm']
del df_stock['stock splits']
del df_stock['rsi']
del df_stock['close_-1_s']
del df_stock['close_-1_d']
del df_stock['rs_14']
